# Illustration of the sensitivity of the flux-variability biomarker prediction on PKU 
Original publication: http://doi.org/10.1038/msb.2009.22

Thierry D.G.A. Mondeel, Vivian Ogundipe & Hans V. Westerhoff <br>
Unversity of Amsterdam  <br>
October 2017

Our implementation of the flux-variability biomarker prediction method allows for alteration of various settings and parameters. Here we show how these settings in the biomarker prediction algorithm lead to changing predictions. 

---

We start by importing Recon 1 and 2.2 and define the proteinogenic amino acids as the potential biomarkers to search for.

In [1]:
import cobra
import pandas as pd
pd.set_option('display.max_colwidth', -1)

import xlrd

from findBiomarkers import findBiomarkers


# import Recon 1 and 2.2
M = cobra.io.load_json_model("../data/RECON1.json") # from BiGG
M22 = cobra.io.load_json_model("../data/RECON_2_2.json") # from the Recon 2.2 publication
M22.id = 'RECON2'

for r in M22.reactions:
    if '_lp_' in r.id:
        s = r.id.replace('_lp','')
        s = s.replace('_rp_','')
        # print r.id,s
        r.id = s
    
M22.repair()

# List of amino acid exhange reactions in RECON 1 and 2 (the IDs differ)
essentialAA = ['EX_his__L_e','EX_ile__L_e','EX_leu__L_e','EX_lys__L_e','EX_met__L_e','EX_phe__L_e',
               'EX_thr__L_e','EX_trp__L_e','EX_val__L_e']
nonEssentialAA = ['EX_cys__L_e','EX_glu__L_e','EX_tyr__L_e','EX_ala__L_e','EX_asp__L_e','EX_gly_e',
                  'EX_arg__L_e','EX_gln__L_e','EX_pro__L_e','EX_ser__L_e','EX_asn__L_e']
rxnsOfInterest = [essentialAA,nonEssentialAA]
rxnsOfInterest = [ item for sublist in rxnsOfInterest for item in sublist ]

essentialAA2 = ['EX_his_L_e','EX_ile_L_e','EX_leu_L_e','EX_lys_L_e','EX_met_L_e','EX_phe_L_e',
               'EX_thr_L_e','EX_trp_L_e','EX_val_L_e']
nonEssentialAA2 = ['EX_cys_L_e','EX_glu_L_e','EX_tyr_L_e','EX_ala_L_e','EX_asp_L_e','EX_gly_e',
                  'EX_arg_L_e','EX_gln_L_e','EX_pro_L_e','EX_ser_L_e','EX_asn_L_e']
rxnsOfInterest2 = [essentialAA2,nonEssentialAA2]
rxnsOfInterest2 = [ item for sublist in rxnsOfInterest2 for item in sublist ]

## Finding appropriate reactions to inactivate for PKU in Recon 1 and Recon 2.2
we construct dictionaries that map IEMs to genes in the three models.

Shlomi et al. included as supplementary information with their publication an Excel file that links IEMs to reactions.

In [2]:
IEMinfo = pd.read_excel('../data/IEM_compendium.xlsx', sheet_name='mappedIEMs supple table', header=0)

# make a dictionary 
IEMdict = dict(zip(IEMinfo['disease (Mapped IEMs)'],IEMinfo['EntrezGeneID'] )) 
        
# get the actual gene string names from Recon1
for iem in IEMdict:
    genelist = IEMdict[iem]
    if type(genelist) is not list:
        genelist = [genelist]

    IEMdict[iem] = list(set([ x for x in [gene.id for gene in M.genes] for y in genelist if x.find(str(y)+'_') != -1 ]))


In Recon 1, PKU is associated with gene ID

In [3]:
IEMdict['phenylketonuria']

['5053_AT1']

Mapped to Recon 2, this ID is:

In [4]:
mapping_recon2_genes = pd.read_csv('../data/hgnc_to_entrez.txt',sep='\t')
mapping_recon2_genes = dict(zip(mapping_recon2_genes['Entrez Gene ID'],mapping_recon2_genes['HGNC ID']))
mapping_recon2_genes[5053]

'HGNC:8582'

The reactions these are associated with in Recon 1 and 2 are

In [5]:
R1_rxns = [r.id for r in M.genes.get_by_id('5053_AT1').reactions]
R2_rxns = [r.id for r in M22.genes.get_by_id('HGNC:8582').reactions]

R1_rxns, R2_rxns

(['PHETHPTOX2'], ['r0399', 'PHETHPTOX2'])

## Set the medium to 1 inward and 1000 outward for all exchanges

In [6]:
# Note: exchange reactions are pseudo-reactions of the form: 'X_e <=> '
exchanges = [ rxn for rxn in M.reactions if len(rxn.products) == 0]
exchanges2 = [ rxn for rxn in M22.reactions if len(rxn.products) == 0]

# all inward to 1
# all outward to 1000
for rxn in exchanges:
    rxn.lower_bound = -1
for rxn in exchanges:
        rxn.upper_bound = 1000
        
for rxn in exchanges2:
    rxn.lower_bound = -1
for rxn in exchanges2:
        rxn.upper_bound = 1000
        
M22.medium;
len(exchanges), len(exchanges2)

(430, 746)

## Sensitivity to network RECON 1 vs. RECON 2.2
Recon 1 finds both Tyrosine and Phenylalanine

In [ ]:
model = M.copy()
T1 = findBiomarkers(model,fvaRxns=rxnsOfInterest,mods=R1_rxns)
T1

Recon 2.2 does not find Tyrosine due to bad gene-mapping

In [ ]:
model = M22.copy()
T = findBiomarkers(model,fvaRxns=rxnsOfInterest2,mods=R2_rxns)
T

## Sensitivity to blocking reactions in sequence or simultaneously
In synchronous mode Recon 2.2 does find Tyrosine. Note that in the mutant Tyrosine may still be produced. 

In [ ]:
model = M22.copy()
T = findBiomarkers(model,fvaRxns=rxnsOfInterest2,mods=R2_rxns,synchronous=True)
T

## Sensitivity to medium choice
When blocking uptake of 34hpp the Tyrosine production in the mutant disappears.

In [ ]:
model = M22.copy()
model.reactions.EX_34hpp_.lower_bound = 0

T = findBiomarkers(model,fvaRxns=rxnsOfInterest2,mods=R2_rxns,synchronous=True)
T

When increasing the 34hpp uptake rate Tyrosine slips below the cutoff. We here show this by lowering the threshold using the appropriate parameter in our implementation.

In [ ]:
model = M22.copy()
model.reactions.EX_34hpp_.lower_bound = -20
T = findBiomarkers(model,fvaRxns=rxnsOfInterest2,mods=R2_rxns,threshold=0.01,synchronous=True)
T